<a href="https://colab.research.google.com/github/Jake-BS/dissertation_code/blob/main/Yolo_auto_trainer_adapted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Config options**

`backupDir` Set to a folder in your gdrive so that trained weights are saved even if the runtime times out.

`resume` Continue training based on any existing weights files in `backupDir`.

`color` Disable hue augmentation if you want the network to use colour as part of the classification decisions.

`tiny` Use the tiny version of yolo. Faster to train and run but less accurate.

`yolo` Set yolo version to use, 3 or 4 supported at present.

In [ ]:
resume = False             # resume training that was previously stopped
color = True              # use color for classification
tiny = True               # train tiny yolo model
yolo = 4                  # yolo version to use
trainProp = 0.6           # proportion of images to use for training
testProp = 0.2
multiplier = 1            # training multiplier, train beyond 'recommeneded' 
colabFree = False         # if using a free version of colab this might be required

datasetFile = "/content/gdrive/MyDrive/Dissertation/the_cones.tar.gz"
backupDir = "/content/gdrive/MyDrive/Dissertation/backuponerun"
classesFile = "/content/gdrive/MyDrive/Dissertation/classes.txt"

# You probably don't need to touch anything below this line
rebuild = False           # recompile darknet, set to true if you have changed any of the options below
size = None, None         # change network input size, set as None, None to use the default

if yolo == 4: size = 416, 416

# try it out
#size = 416, 736

# input size must be multiple of 32
assert( size[0]%32 == 0 )
assert( size[1]%32 == 0 )

validProp = 1 - trainProp - testProp

gpu_info = !nvidia-smi   # automatically detect if you are using a gpu runtime
gpu = '\n'.join(gpu_info).find('failed') == -1

tensor = True
opencv = True
cudnn = True

import os

wrkDir = "/content"

tempDir = os.path.join( wrkDir, "temp" )
dataDir = os.path.join( wrkDir, "data" )
yoloDir = os.path.join( wrkDir, "yolo" )
darknetDir = os.path.join( wrkDir, "darknet" )
trainingDir = os.path.join(yoloDir, "training")

dataFile = os.path.join(yoloDir, "obj.data")
namesFile = os.path.join(yoloDir, "obj.names")
trainingFile = os.path.join(yoloDir, "train.txt")
testingFile = os.path.join(yoloDir, "test.txt")
validFile = os.path.join(yoloDir, "valid.txt")
trainingCfgFile = os.path.join(yoloDir, "training.cfg")
trainedCfgFile = os.path.join(yoloDir, "trained.cfg")
weightsFile = os.path.join(yoloDir,"yolo.weights")
trainedWeightsFile = os.path.join( backupDir, "training_best.weights" )


In [ ]:
# display config options
print( f"colabFree  = {colabFree}" )
print( f"color      = {color}" )
print( f"cudnn      = {cudnn}" )
print( f"gpu        = {gpu}" )
print( f"multiplier = {multiplier}" )
print( f"opencv     = {opencv}" )
print( f"rebuild    = {rebuild}" )
print( f"resume     = {resume}" )
print( f"size       = {size}" )
print( f"tensor     = {tensor}" )
print( f"testProp   = {testProp}" )
print( f"tiny       = {tiny}" )
print( f"trainProp  = {trainProp}" )
print( f"validProp  = {validProp}" )
print( f"yolo       = {yolo}" )
print()

print( f"tempDir     = {tempDir}" )
print( f"dataDir     = {dataDir}" )
print( f"yoloDir     = {yoloDir}" )
print( f"darknetDir  = {darknetDir}" )
print( f"trainingDir = {trainingDir}" )
print( f"backupDir   = {backupDir}" )
print()

print( f"dataFile           = {dataFile}" )
print( f"namesFile          = {namesFile}" )
print( f"trainingFile       = {trainingFile}" )
print( f"testingFile        = {testingFile}" )
print( f"trainingCfgFile    = {trainingCfgFile}" )
print( f"trainedCfgFile     = {trainedCfgFile}" )
print( f"trainedWeightsFile = {trainedWeightsFile}" )
print( f"validFile          = {validFile}" )
print( f"weightsFile        = {weightsFile}" )

colabFree  = False
color      = True
cudnn      = True
gpu        = True
multiplier = 1
opencv     = True
rebuild    = False
resume     = False
size       = (416, 416)
tensor     = True
testProp   = 0.2
tiny       = True
trainProp  = 0.6
validProp  = 0.2
yolo       = 4

tempDir     = /content/temp
dataDir     = /content/data
yoloDir     = /content/yolo
darknetDir  = /content/darknet
trainingDir = /content/yolo/training
backupDir   = /content/gdrive/MyDrive/Dissertation/backuponerun

dataFile           = /content/yolo/obj.data
namesFile          = /content/yolo/obj.names
trainingFile       = /content/yolo/train.txt
testingFile        = /content/yolo/test.txt
trainingCfgFile    = /content/yolo/training.cfg
trainedCfgFile     = /content/yolo/trained.cfg
trainedWeightsFile = /content/gdrive/MyDrive/Dissertation/backuponerun/training_best.weights
validFile          = /content/yolo/valid.txt
weightsFile        = /content/yolo/yolo.weights


# Mount your google drive, and extract the contents of your dataset .zip

In [ ]:
        #mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
#!ln -s "/content/gdrive/My Drive/" /mydrive

/
Mounted at /content/gdrive


In [ ]:
!rm -r {tempDir}
!mkdir -p {tempDir}
#!unzip -qo {datasetFile} -d {tempDir}
!tar -xf {datasetFile} --directory {tempDir}
!ls {tempDir}

rm: cannot remove '/content/temp': No such file or directory
the_cones


## Move all image and text files from the extract directory into the data directory.

So we don't have to deal with different directory structures going forwards

In [ ]:
import glob, os, shutil

try: shutil.rmtree( dataDir )
except FileNotFoundError: pass

os.makedirs( dataDir )

# find and relocate all images
print( f"Move contents of {tempDir}")

for ext in ('*.png', '*.jpg', "*.txt"):
  globPath = os.path.join( tempDir, "**", ext )
  print( globPath )
  for filename in glob.glob( globPath, recursive=True ):
    print( filename )
    shutil.copyfile( filename, os.path.join( dataDir, os.path.split(filename)[-1] ) )

Move contents of /content/temp
/content/temp/**/*.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left001740.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002240.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left003100.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left003300.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002340.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left003240.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_many_left000020.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002320.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002220.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002360.png
/content/temp/the_cones/05_david_done/HD1080_zed_wet_behind_ecb_few_left002620.png
/content/temp/the_cones/05_david

## Check dataset is valid

- Classes file is present and readable.
- Images are jpgs (convert if needed).
- Images have matching labels.
- Label files are correctly formatted.


In [ ]:
# name your classes
#classes = ["blue","orange","yellow"]
with open( os.path.join(dataDir, classesFile), "r" ) as f:
  classes = [ i.strip() for i in f ]

print( classes )

['blue', 'orange', 'yellow']


In [ ]:
#convert pngs to jpg
#!apt install -y imagemagick

import os, glob, re
import cv2

# convert pngs
for filename in glob.glob( f"{dataDir}/*.png" ):
  print( f"Convert {filename}" )
  #os.system( f'convert "{filename}" "{filename.replace(".png",".jpg")}"' )
  #os.remove( filename )
  image = cv2.imread( filename )
  if image is None:
    print( f"Failed to open {filename}" )
  else:
    cv2.imwrite( ".".join(filename.split(".")[:-1])+".jpg", image )
  os.remove( filename )

# check label file exists
for filename in glob.glob( f"{dataDir}/*.jpg" ):
  if not os.path.exists( filename.replace( ".jpg", ".txt" ) ):
    print( f"Missing labels {filename}" )
    os.remove( filename )

# confirm images are valid
#for filename in glob.glob( f"{dataDir}/*.jpg" ):
#  result = os.system( f"identify -format '%f' '{filename}'" )
#  if result:
#      print( f"Identify fail {filename}" )
      #os.remove( filename )

# confirm that label files are in correct format
yoloReg = re.compile( r"^(\s*([0-9]{1,})\s{1,}(\S{1,})\s{1,}(\S{1,})\s{1,}(\S{1,})\s{1,}(\S{1,})\s*|\s*)$" )
for filename in glob.glob( f"{dataDir}/*.txt" ):
  if filename.endswith( classesFile ): continue

  valid = True
  with open( filename, "r" ) as f:
    for line in f:
      matches = yoloReg.match( line ).groups()
      
      c = int(matches[1])
      if c<0 or c>=len(classes): valid = False
      else:
        for i in matches[-4:]:
          i = float(i)
          if i<0 or i>1: valid = False

  if not valid:
    print( f"{filename} is invalid" )
    os.remove( filename )
    os.remove( filename.replace(".txt", ".jpg") )


Convert /content/data/HD1080_zed2_cloud_road_few_left000120.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left001740.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left002240.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left003100.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left000600.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left000720.png
Convert /content/data/HD1080_zed2_sunny_behind_ecb_few_left001080.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left003300.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left002340.png
Convert /content/data/HD1080_zed2_sunny_behind_ecb_few_left000860.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left003240.png
Convert /content/data/HD1080_zed_wet_behind_ecb_many_left000020.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left002320.png
Convert /content/data/HD1080_zed_wet_behind_ecb_few_left000480.png
Convert /content/data/HD1080_zed2_sunny_library_few_left00

# **Clone `darknet` git repository** 

Needed now as we are about to start generating the configuration files based on the default ones


In [ ]:
if not os.path.exists( darknetDir ) or rebuild:
  try: shutil.rmtree( darknetDir )
  except FileNotFoundError: pass

  !git clone https://github.com/AlexeyAB/darknet {darknetDir}

  #os.system( f"git clone https://github.com/AlexeyAB/darknet {darknetDir}" )

  #!git clone https://github.com/leggedrobotics/darknet /content/darknet
  #!git clone https://github.com/pjreddie/darknet {darknetDir}

Cloning into '/content/darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 6.60 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


## Create config files


In [ ]:
try: shutil.rmtree( yoloDir )
except FileNotFoundError: pass

os.makedirs( yoloDir )

print( f"Create {dataFile}" )
with open( dataFile, "w" ) as f:
  f.write( f"classes = {len(classes)}\n" )
  f.write( f"train = {trainingFile}\n" )
  f.write( f"test = {testingFile}\n" )
  f.write( f"valid = {validFile}\n" )
  f.write( f"names = {namesFile}\n" )
  f.write( f"backup = {backupDir}\n" )
!cat {dataFile}

print( f"Create {namesFile}" )
with open( namesFile, "w" ) as f:
  for i in classes:
    f.write( f"{i}\n" )

Create /content/yolo/obj.data
classes = 3
train = /content/yolo/train.txt
test = /content/yolo/test.txt
valid = /content/yolo/valid.txt
names = /content/yolo/obj.names
backup = /content/gdrive/MyDrive/Dissertation/backuponerun
Create /content/yolo/obj.names


In [ ]:
# split the images into testing and training groups
# assuming that similar images have similar files names, this approach ensures
# a representative distribution between the groups and will produce the same
# split every time

import math, os

files = sorted( [ i for i in os.listdir( dataDir ) if i.endswith(".jpg") ] )

buckets = ( [ open( trainingFile, "w" ), 0, trainProp ],
			      [ open( testingFile, "w" ), 0, testProp ], 
			      [ open( validFile, "w" ), 0, validProp ] )

for img in files:
  s = sorted( buckets, key=lambda i: i[1] / (len(files)*i[2]) if i[2] else math.inf )

  s[0][1] += 1
  s[0][0].write( f"{os.path.join(dataDir,img)}\n" )

for f, count, _ in buckets:
  print( count )
  f.close()


459
154
153


## **Generate network config files**



In [ ]:
import os, re

# how many training images have we got
try:
  with open(trainingFile, 'r') as f:
    images = sum( ( 1 for i in f if i.strip() != "" ) )
except:
  images = 1000

sourceFiles = { (3,True):  os.path.join( darknetDir, "cfg", "yolov3-tiny.cfg" ),
                (3,False): os.path.join( darknetDir, "cfg", "yolov3.cfg" ),
                (4,True):  os.path.join( darknetDir, "cfg", "yolov4-tiny-custom.cfg" ),
                (4,False): os.path.join( darknetDir, "cfg", "yolov4-custom.cfg" ) }
 
#maxBatches = max(len(classes)*2000,images,6000) * multiplier
maxBatches = 6000
steps = int(maxBatches*0.8), int(maxBatches*0.9)

blur=1
  
momentum=0.9

decay=0.0005

learning = 0.003

#width = 736
#height = 736

replacements = [ 
  ( r"^\s*batch\s*=\s*[0-9]{1,}",        f"batch={[64,32][colabFree]}\n" ),
  ( r"^\s*subdivisions\s*=\s*[0-9]{1,}", f"subdivisions={[16,8][colabFree]}\n" ),
  ( r"^\s*max_batches\s*=\s*[0-9]{1,}", f"max_batches={maxBatches}\n" ),
  ( r"^\s*steps\s*=\s*[0-9,]{3,}",      f"steps={steps[0]},{steps[1]}\n" ),
  ( r"^\s*classes\s*=\s*[0-9]{1,}",     f"classes={len(classes)}\n" ),
  ( r"^\s*filters\s*=\s*255",           f"filters={(len(classes)+5)*3}\n" ),
  #My replacements are below
  #( r"^\s*width\s*=\s*[0-9]{1,}", f"width={width}\n" ),
  #( r"^\s*height\s*=\s*[0-9]{1,}", f"height={height}\n" ),
  #Below parameters have decimal values so added .[0-9] after [0-9]
  ( r"^\s*momentum\s*=\s*[0-9].[0-9]{1,}", f"momentum={momentum}\n" ),
  ( r"^\s*decay\s*=\s*[0-9].[0-9]{1,}", f"decay={decay}\n" ),
  ( r"^\s*learning_rate\s*=\s*[0-9].[0-9]{1,}", f"learning_rate={learning}\nblur={blur}\n" ),
  #( r"^\s*learning_rate\s*=\s*[0-9].[0-9]{1,}", f"learning_rate={learning}\nblur={blur}\nflip=1\nmosaic=1\nmixup=1\n" ),
    ]

altReplacements = [
  ( r"^\s*batch=64", "batch=1\n" ),
  ( r"^\s*subdivision=16", "subdivision=1\n" )
  ]


# will colour be used for discrimination?
'''
if color:
  replacements.append( ( r"^hue=\.1", "hue=0\n" ) )
'''

if size[0] != None:
  assert size[0] % 32 == 0
  replacements.append( ( r"^\s*height\s*=\s*[0-9]{1,}",
                         f"height={int(size[0]/32)*32}\n" ) )

if size[1] != None:
  assert size[1] % 32 == 0
  replacements.append( ( r"^\s*width\s*=\s*[0-9]{1,}",
                         f"width={int(size[1]/32)*32}\n" ) )

replacements = [ ( re.compile( i ), j ) for i, j in replacements ]
altReplacements = [ ( re.compile( i ), j ) for i, j in altReplacements ]

sourceFilename = sourceFiles[ (yolo,tiny) ]
print( f"Using {sourceFilename} as source file ")
with open( sourceFilename, "r" ) as i:
  with open( trainingCfgFile, "w" ) as o:
    with open( trainedCfgFile, "w" ) as o2:

      for line in i:
        before = line
        for reg, rep in replacements:
          if reg.match( line ):
            line = rep
        o.write( line )

        if line != before: print( line, end="" )

        for reg, rep in altReplacements:
          if reg.match( line ):
            line = rep
        o2.write( line )


Using /content/darknet/cfg/yolov4-tiny-custom.cfg as source file 
subdivisions=16
learning_rate=0.003
blur=1
max_batches=6000
steps=4800,5400
filters=24
classes=3
filters=24
classes=3


# **Modify the darknet `makefile`**

We are going to enable OpenCV and GPU processing for better performance.

In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

replacements = [ 
  ( r"^\s*LIBSO=0",       "LIBSO=1\n" )
   ]

if opencv:
  replacements.append( ( r"OPENCV=0", "OPENCV=1\n" ) )

if gpu:
  replacements.append( ( r"^\s*GPU=0", "GPU=1\n" ) )

  if cudnn:
    replacements.append( ( r"^\s*CUDNN=0",       "CUDNN=1\n" ) )

  if tensor:
    replacements.append( ( r"^\s*CUDNN_HALF=0",  "CUDNN_HALF=1\n" )  )

replacements = [ ( re.compile( i ), j ) for i, j in replacements ]

print( f"Edit {darknetDir}" )

with open( os.path.join( darknetDir, "Makefile" ), "r" ) as i:
  contents = i.readlines()

with open( os.path.join( darknetDir, "Makefile" ), "w" ) as o:
  for line in contents:
    #print( line )
    for reg, rep in replacements:
      if reg.match( line ):
        line = rep
    o.write( line )

Edit /content/darknet


# **Build darknet**

In [ ]:
# build darknet 
if not os.path.exists( os.path.join( darknetDir, "darknet" ) ) or rebuild:
  #os.system( f"make -C {darknetDir}" )
  !make -C {darknetDir}

make: Entering directory '/content/darknet'
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-in

# **Download the pre-trained *`yolo`* weights**

Or resume training on the ones backed up to the gdrive

In [ ]:
import shutil

lastWeightsFile = os.path.join( backupDir, "training_last.weights" )

if not resume or not os.path.exists( lastWeightsFile ): # resumt is False or the last weights don't exist
  print( "Download pre-trained weights" )
  if yolo == 4 and tiny:
    url = "https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29"
  elif yolo == 4 and not tiny:
    url = "https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"
  elif yolo == 3 and not tiny:
    url = "https://pjreddie.com/media/files/darknet53.conv.74"

  print( url )
  os.system( f"wget {url} -O {weightsFile}" )
  %cd {darknetDir}
  #If you want to train from the beginning, then use flag in the end of training command: -clear 
  cmd = f"./darknet detector train {dataFile} {trainingCfgFile} {weightsFile} -dont_show -map -clear | tee -a {os.path.join(backupDir, 'darknet.log')}"
  print( cmd )
  !{cmd}
else:
  print( "Resuming" )

  shutil.copyfile( lastWeightsFile, weightsFile )



Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.933095), count: 1, class_loss = 0.000003, iou_loss = 0.208028, total_loss = 0.208031 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.615822), count: 28, class_loss = 0.037054, iou_loss = 310.864288, total_loss = 310.901337 
 total_bbox = 2992704, rewritten_bbox = 13.332458 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.889717), count: 16, class_loss = 0.001397, iou_loss = 111.704811, total_loss = 111.706207 
 total_bbox = 2992720, rewritten_bbox = 13.332387 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.0

## **Train your custom detector** 

For best results, you should stop the training when the average loss is less than 0.05 if possible or at least below 0.3, else train the model until the average loss does not show any significant change for a while.

In [ ]:
  import os

  !mkdir -p {backupDir}
  !cp -r {yoloDir} {backupDir}/.
  
  %cd {darknetDir}

  cmd = f"./darknet detector train {dataFile} {trainingCfgFile} {weightsFile} -dont_show -map -clear | tee -a {os.path.join(backupDir, 'darknet.log')}"
  print( cmd )
  !{cmd} 

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.937412), count: 1, class_loss = 0.000001, iou_loss = 0.210580, total_loss = 0.210581 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.458212), count: 63, class_loss = 1.321537, iou_loss = 311.021973, total_loss = 312.343506 
 total_bbox = 2998325, rewritten_bbox = 13.376602 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.937103), count: 1, class_loss = 0.000005, iou_loss = 0.210345, total_loss = 0.210350 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.637880), count: 30, class_loss = 0.561900, iou_loss = 397.784393, total_loss = 398.346283 
 total_bbox = 2998356, rewritten_bbox = 13.376631 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.0

In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False

AssertionError: ignored

## **Use this simple hack for Auto-Click to avoid being kicked off Colab VM**

Press (Ctrl + Shift + i) . Go to console. Paste the following code and press Enter.

```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```


# **Check performance** 



In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


**Check the training chart**

In [ ]:
#only works if the training does not get interrupted
imShow('chart.png')

**Check mAP (mean average precision)**

In [ ]:
!./darknet detector map {dataFile} {trainedCfgFile} {trainedWeightsFile} 2>&1 | egrep "class_id|conf_thresh|IoU|mean average"


# **Test your custom Object Detector**

## **Run detector on an image**

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

import os, random

files = [ i.strip() for i in open(testingFile, "r") ]
filename = random.choice( files )

#filename = "/content/yolov4/data/obj/1925.jpg"
print( filename )

%cd {darknetDir}

trainedWeightsFile = os.path.join( backupDir, "training_best.weights" )

cmd = f"./darknet detector test {dataFile} {trainedCfgFile} {trainedWeightsFile} {filename} -thresh 0.5 2>&1 >/dev/null"
print( cmd )
!{cmd} 

from IPython.display import Image
Image( os.path.join(darknetDir,'predictions.jpg') )